# Data Extractor

In [1]:
import ee
ee.Initialize()
ee.Authenticate()
import geetools
import geemap
import os

Enter verification code: 4/1AdQt8qh2sCr0J0Z4EjUonkVf9sGjjiDEUz80N7WqFD3M1YN04FA-yGXWprg

Successfully saved authorization token.


In [9]:
# Parameters.
data_collection = "COPERNICUS/S2_SR_HARMONIZED"
# Time windows.
start_date = "2021-08-01"
end_date = "2021-08-20"
# Color bands.
bands = ['B4', 'B3', 'B2']
# Coordinates
coordinates = {
    "Germany" : {
        "p1" : [9.368331, 52.409899],
        "p2" : [10.041240, 52.489424],
        "p3" : [10.109178, 52.295949],
        "p4" : [9.575380, 52.244477],
    }
}

In [10]:
# Create a polygon based on a JSON entry.
polygonGeoJSON = ee.Geometry.Polygon(
    [
        # Outside ring -> [lat, long]
        [
            coordinates["Germany"]["p1"],
            coordinates["Germany"]["p2"],
            coordinates["Germany"]["p3"],
            coordinates["Germany"]["p4"],
            coordinates["Germany"]["p1"],
        ],
    ]
)

In [11]:
Map = geemap.Map()
Map.addLayer(polygonGeoJSON, {}, 'polygonGeoJSON')
Map.centerObject(polygonGeoJSON)

In [12]:
Map

Map(center=[52.3669505, 9.738754499999999], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

In [13]:
image_collection = (
    ee.ImageCollection(data_collection)
    .select(bands)
    .filterBounds(polygonGeoJSON)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
)

In [14]:
print(f'The size of the collection is: {image_collection.size().getInfo()}')

The size of the collection is: 1


In [15]:
visualization = {
    "bands" : bands,
    "min": 300, 
    "max": 3500
}

Map.addLayer(image_collection, visualization, 'RGB')

In [16]:
#for band in bands:
tasks = geetools.batch.Export.imagecollection.toDrive(
    collection=image_collection.select(bands), 
    folder='ClimateData',
    verbose=True,
    region=polygonGeoJSON)

exporting 20210814T102031_20210814T102609_T32UNC to folder 'ClimateData' in GDrive
